# 1xbet_offensive 데이터를 기준으로
## 연도, 선수 이름, (해당 시즌 기준 이적 후)팀, 포지션 정보를 가지는 테이블 생성
## 고유번호, 해당 년도 나이를 입력을 추가

In [1]:
import pandas as pd

In [2]:
# offensive data csv파일 받아오기
offensive_path = './data/new/1xbet_offensive_edited/1xbet_offensive_{}_edited.csv'

start_year, end_year = 2014, 2022
stats_df = pd.DataFrame()

for year in range(start_year, end_year+1):
    o_df = pd.read_csv(offensive_path.format(year))
    o_df['Year'] = year
    stats_df = pd.concat([stats_df, o_df])

In [3]:
stats_df.head()

,Name,Team,Age,Position,Apps,Mins,Goals,Assists,SpG,KeyP,Drb,Fouled,Off,Disp,UnsTch,Rating,Year
0,Eden Hazard,Chelsea,32,Forward,38,3379,14,9,2.052632,2.631579,4.763158,2.973684,0.105263,2.710526,2.105263,7.956842,2014
1,Alexis Sanchez,Arsenal,34,Forward,35,2953,16,8,3.485714,2.342857,3.285714,2.057143,0.228571,3.314286,2.114286,7.810857,2014
2,Sergio Aguero,Man City,35,Forward,33,2540,26,8,4.484848,1.000000,2.636364,0.757576,1.000000,2.727273,1.939394,7.671515,2014
3,Cesc Fabregas,Chelsea,36,Midfielder,34,2890,3,18,1.264706,2.794118,1.058824,1.058824,0.029412,1.411765,1.000000,7.618529,2014
4,Santi Cazorla,Arsenal,38,Midfielder,37,2992,7,11,2.513514,2.108108,2.351351,1.648649,0.054054,1.486486,1.000000,7.599459,2014


In [4]:
# 시즌 도중 이적하여 한 시즌에 중복 데이터가 들어있는 선수 예시
stats_df[stats_df['Name'] == 'Theo Walcott'][['Name', 'Team', 'Year']]

,Name,Team,Year
339,Theo Walcott,Arsenal,2014
348,Theo Walcott,Arsenal,2015
111,Theo Walcott,Arsenal,2016
49,Theo Walcott,Everton,2017
494,Theo Walcott,Arsenal,2017
322,Theo Walcott,Everton,2018
353,Theo Walcott,Everton,2019
288,Theo Walcott,Southampton,2020
522,Theo Walcott,Everton,2020
499,Theo Walcott,Southampton,2021


In [5]:
# 선수 정보만 가져온 테이블 - 시즌 중 이적한 선수의 데이터를 이적한 팀 기준으로 정리해야 함
player_table = stats_df[['Year', 'Name', 'Age', 'Team', 'Position']]
# 인덱스 리셋
player_table = player_table.reset_index()

# 비교를 위한 원본 데이터 유지
origin_player = player_table

In [6]:
# 정리해야 할 선수 목록 선별
# 선수별로 Year의 고유값 갯수가 데이터 갯수 보다 작은 경우

# 고유 이름
names = player_table.Name.unique()
# 이름 순회하며 Year갯수와 Name갯수를 비교
dupli_names = [] # 정리해야 할 선수 목록
for name in names:
    player = player_table[player_table['Name'] == name]
    if (len(player['Year'].unique()) < player['Name'].count()):
        dupli_names.append(name)

정리해야 하는 선수들의 경우  
"문제가 있는 년도 팀명 --> 2개  
이전 혹은 이후 년도 팀명 --> 1개"  
의 구조를 따른다.  
이전, 이후 년도 팀명 중 하나를 사용하여 문제가 있는 년도의 팀명을 통합

In [7]:
# 이전 팀명과 비교
def compare_before(target, name):
    # 타켓 년도, 선수
    player = player_table[(player_table['Year'] == target) & (player_table['Name'] == name)]
    
    # 비교할 이전 시즌이 있는 경우
    years = player_table[player_table['Name'] == name]['Year'].unique()
    if (target - 1) in years:
        # 비교할 이전 시즌 팀명
        compare_team = player_table[(player_table['Year'] ==  (target - 1)) \
                                            & (player_table['Name'] == name)]['Team'].unique()[0]
        # 타켓 년도 두 개의 팀명
        target_team1 = player['Team'].unique()[0]
        target_team2 = player['Team'].unique()[1]

        # 작년 소속팀, 올해 두 개의 소속팀이 모두 다르다면 내년 팀과 비교
        if target_team1 != compare_team != target_team2:
            return compare_after(target, name)

        # 작년 소속팀과 같으면 인덱스 저장 (삭제를 위해)
        if target_team1 == compare_team:
            idx = player.index.unique()[0]
            return idx

        if target_team2 == compare_team:
            idx = player.index.unique()[1]
            return idx
    else:
        idx = player.index.unique()[0]
        return idx

In [8]:
# 이후 팀명과 비교
def compare_after(target, name):    
    # 타켓 년도, 선수
    player = player_table[(player_table['Year'] == target) & (player_table['Name'] == name)]
    # 비교할 이후 시즌이 있는 경우
    years = player_table[player_table['Name'] == name]['Year'].unique()
    if (target + 1) in years:
        # 비교할 이후 시즌 팀명
        compare_team = player_table[(player_table['Year'] ==  (target + 1)) \
                                            & (player_table['Name'] == name)]['Team'].unique()[0]

        # 타켓 년도 두 개의 팀명   
        target_team1 = player['Team'].unique()[0]
        target_team2 = player['Team'].unique()[1]

        # 내년 소속팀과 다르면 인덱스 저장 (삭제를 위해)
        if target_team1 != compare_team:
            idx = player.index.unique()[0]
            return idx

        if target_team2 != compare_team:
            idx = player.index.unique()[1]
            return idx
    else:
        idx = player.index.unique()[0]
        return idx

In [9]:
# 선수 데이터 삭제 함수
def del_player(name):
    # 특정 선수 이름을 통해 중복이 있는 년도를 뽑아 타켓 년도 추출
    player = player_table[player_table['Name'] == name]
    dupli_year = player.groupby('Year')['Name'].count()
    targets = dupli_year[dupli_year > 1].index
    
    idxs = []
    
    # 타켓 년도 별로 선수 데이터 정리
    for target in targets:
        if target == 2014:
            idxs.append(compare_after(target, name))     
        elif target == 2022:
            idxs.append(compare_before(target, name))    
        else:
            idxs.append(compare_before(target, name))
            
    return idxs

In [10]:
# 정리가 필요한 선수 리스트를 순회하며 데이터 정리
for name in dupli_names:
    idxs = del_player(name)
    for idx in idxs:
        player_table = player_table.drop(index = idx)

In [11]:
# 한 시즌에 중복 데이터가 있는 경우 체크

names = player_table.Name.unique()
# 이름 순회하며 Year갯수와 Name갯수를 비교
dupli_names = []
for name in names:
    player = player_table[player_table['Name'] == name]
    if (len(player['Year'].unique()) < player['Name'].count()):
        dupli_names.append(name)
dupli_names

[]

In [12]:
# 이전 데이터와 현재 데이터 비교
display(origin_player[origin_player['Name'] == 'Danny Welbeck'])
player_table[player_table['Name'] == 'Danny Welbeck']

,index,Year,Name,Age,Team,Position
62,62,2014,Danny Welbeck,32,Arsenal,Forward
420,420,2014,Danny Welbeck,32,Man Utd,Forward
643,94,2015,Danny Welbeck,32,Arsenal,Forward
1431,321,2016,Danny Welbeck,32,Arsenal,Forward
1998,345,2017,Danny Welbeck,32,Arsenal,Forward
2558,376,2018,Danny Welbeck,32,Arsenal,Forward
3070,380,2019,Danny Welbeck,32,Watford,Forward
3443,231,2020,Danny Welbeck,32,Brighton,Forward
3960,216,2021,Danny Welbeck,32,Brighton,Forward
4419,128,2022,Danny Welbeck,32,Brighton,Forward


,index,Year,Name,Age,Team,Position
62,62,2014,Danny Welbeck,32,Arsenal,Forward
643,94,2015,Danny Welbeck,32,Arsenal,Forward
1431,321,2016,Danny Welbeck,32,Arsenal,Forward
1998,345,2017,Danny Welbeck,32,Arsenal,Forward
2558,376,2018,Danny Welbeck,32,Arsenal,Forward
3070,380,2019,Danny Welbeck,32,Watford,Forward
3443,231,2020,Danny Welbeck,32,Brighton,Forward
3960,216,2021,Danny Welbeck,32,Brighton,Forward
4419,128,2022,Danny Welbeck,32,Brighton,Forward


In [13]:
# 년도, 이름, 팀, 포지션이 정리 된 상태
player_table

,index,Year,Name,Age,Team,Position
0,0,2014,Eden Hazard,32,Chelsea,Forward
1,1,2014,Alexis Sanchez,34,Arsenal,Forward
2,2,2014,Sergio Aguero,35,Man City,Forward
3,3,2014,Cesc Fabregas,36,Chelsea,Midfielder
4,4,2014,Santi Cazorla,38,Arsenal,Midfielder
...,...,...,...,...,...,...
4855,564,2022,Dexter Lembikisa,19,Wolves,Defender
4856,565,2022,Cameron Peupion,20,Brighton,Midfielder
4857,566,2022,Andrew Moran,19,Brighton,Midfielder
4858,567,2022,Shane Duffy,31,Fulham,Defender


In [14]:
player_table = player_table.rename(columns = {'Age' : 'Curr_Age'})

In [15]:
# 년도 기준 나이 데이터 생성
player_table['Age'] = player_table['Curr_Age'] - (2023 - player_table['Year'])

In [16]:
# 이름 별로 고유번호 부여하기

# 총 이름 갯수
len(player_table['Name'].unique())

1734

In [17]:
# 이름 순으로 정렬 후 동일한 고유 번호 부여
# 총 이름 갯수와 마지막 고유번호가 동일한 것을 확인할 수 있음
rank_table = pd.DataFrame(player_table['Name'].sort_values())
rank_table['No.'] = rank_table.rank(method = 'dense')
rank_table = rank_table.groupby('Name')['No.'].mean()
rank_df = pd.DataFrame(rank_table.astype('int'))
rank_df

,No.
Name,
Aaron Connolly,1
Aaron Cresswell,2
Aaron Hickey,3
Aaron Lennon,4
Aaron Mooy,5
...,...
Yun Suk-Young,1730
Yves Bissouma,1731
Zack Steffen,1732


In [18]:
# 선수 테이블에 고유번호 병합
player_table = pd.merge(player_table, rank_df, how = 'left', left_on = 'Name', right_on = 'Name')
player_table

,index,Year,Name,Curr_Age,Team,Position,Age,No.
0,0,2014,Eden Hazard,32,Chelsea,Forward,23,450
1,1,2014,Alexis Sanchez,34,Arsenal,Forward,25,74
2,2,2014,Sergio Aguero,35,Man City,Forward,26,1508
3,3,2014,Cesc Fabregas,36,Chelsea,Midfielder,27,261
4,4,2014,Santi Cazorla,38,Arsenal,Midfielder,29,1480
...,...,...,...,...,...,...,...,...
4751,564,2022,Dexter Lembikisa,19,Wolves,Defender,18,411
4752,565,2022,Cameron Peupion,20,Brighton,Midfielder,19,247
4753,566,2022,Andrew Moran,19,Brighton,Midfielder,18,110
4754,567,2022,Shane Duffy,31,Fulham,Defender,30,1515


In [19]:
# 년도 마다 동일한 선수에 동일한 고유번호가 부여 되었음을 확인
player_table[player_table['Name'] == 'Reece James']

,index,Year,Name,Curr_Age,Team,Position,Age,No.
2714,89,2019,Reece James,23,Chelsea,Defender,19,1381
3216,76,2020,Reece James,23,Chelsea,Defender,20,1381
3684,20,2021,Reece James,23,Chelsea,Defender,21,1381
4229,27,2022,Reece James,23,Chelsea,Defender,22,1381


In [20]:
# 컬럼 순서 변경 및 정리

player_table = player_table[['No.', 'Year', 'Name', 'Age', 'Team', 'Position']]
player_table.rename(columns = {'Year' : 'year'}, inplace = True)
player_table

/var/folders/5r/gwhqhzlx7g5_xdjhhpzr42_h0000gn/T/ipykernel_47471/2500907302.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  player_table.rename(columns = {'Year' : 'year'}, inplace = True)


,No.,year,Name,Age,Team,Position
0,450,2014,Eden Hazard,23,Chelsea,Forward
1,74,2014,Alexis Sanchez,25,Arsenal,Forward
2,1508,2014,Sergio Aguero,26,Man City,Forward
3,261,2014,Cesc Fabregas,27,Chelsea,Midfielder
4,1480,2014,Santi Cazorla,29,Arsenal,Midfielder
...,...,...,...,...,...,...
4751,411,2022,Dexter Lembikisa,18,Wolves,Defender
4752,247,2022,Cameron Peupion,19,Brighton,Midfielder
4753,110,2022,Andrew Moran,18,Brighton,Midfielder
4754,1515,2022,Shane Duffy,30,Fulham,Defender


In [21]:
# 년도 별로 csv 파일 저장
start_year, end_year = 2014, 2022

for year in range(start_year, end_year+1):
    table = player_table[player_table['year'] == year]
    table.to_csv(path_or_buf = f'./data/new/players/players_{year}.csv', encoding = 'utf-8')

In [22]:
# 전체 데이터 csv 파일 저장
player_table.to_csv(path_or_buf = f'./data/new/players_all.csv', encoding = 'utf-8')